# Лабораторная работа 6. Линейная регрессия

## <a href="https://www.kaggle.com/datasets/canggih/anime-data-score-staff-synopsis-and-genre">Anime Data с сайта MyAnimeList.net</a>

### Описание набора данных:
Японская анимация, известная как аниме, в наши дни получила международное распространение. В этом наборе данных представлены данные об аниме, взятые из MyAnimeList.

### Прилогающаяся информация:
Этот набор данных можно использовать для создания систем рекомендаций, прогнозирования результатов, визуализации сходства с аниме и т.д.


1) Title - <b>Категориальное</b> - Название
2) Type - <b>Категориальное</b> - Тип аниме (TV/Movie/OVA/ONA/Special)
3) Episodes - <b>Целое</b> - Количество эпизодов
4) Status - <b>Категориальное</b> - Статус ('Finished Airing', 'Currently Airing')
5) Start airing - <b>Категориальное</b> - Начало показа ('YYYY-MM-DD')
6) End airing - <b>Категориальное</b> - Конец показа ('YYYY-M-D')
7) Starting season - <b>Категориальное</b> - Время года показа ('Fall', 'Summer',  'Winter', 'Spring')
8) Broadcast time - <b>Категориальное</b> - Время трансляции ('Day of the week' at 'HH:MM' (JST))
9) Producers - <b>Категориальное</b> - Продюссер
10) Licensors - <b>Категориальное</b> - Лицензия
11) Studios - <b>Категориальное</b> - Студия
12) Sources - <b>Категориальное</b> - Первоисточник ('Manga', 'Original', 'Light, 'novel', 'Novel')
13) Genres - <b>Категориальное</b> - Жанр
14) Duration - <b>Категориальное</b> - Продолжительность
15) Rating - <b>Категориальное</b> - Возрастной рейтинг ('G', 'PG-13', 'PG', 'R')
16) Score - <b>Дробное</b> - Средняя оценка  
17) Scored by - <b>Целое</b> - Количество оценок 
18) Members - <b>Целое</b> - Число подписчиков
19) Favorites - <b>Целое</b> - В избранном
20) Description - <b>Категориальное</b> - Описание

Подключение библиотек

In [4]:
import numpy as np
import pandas as pd
import scipy
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error


%matplotlib inline

Чтение исходного набора данных

In [5]:
ds_anime = pd.read_csv('dataanime.csv')
test_anime = pd.read_csv('dataanime.csv', nrows=10)

print(ds_anime)
print('/' * 20)
print(test_anime)

                                                  Title   Type Episodes  \
0                      Fullmetal Alchemist: Brotherhood     TV       64   
1                                        Kimi no Na wa.  Movie        1   
2                                              Gintama°     TV       51   
3                                         Steins;Gate 0     TV       23   
4                                           Steins;Gate     TV       24   
...                                                 ...    ...      ...   
1558                                    Kuroshitsuji II     TV       12   
1559                              Minami-ke Natsuyasumi    OVA        1   
1560  Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsu...  Movie        1   
1561        Persona 4 the Animation: The Factor of Hope  Movie        1   
1562                                       Re:␣Hamatora     TV       12   

                Status Start airing End airing Starting season  \
0      Finished Airing     2009-4

In [6]:
print(ds_anime.head(1).T)
print('/' * 20)
print(test_anime.head(1).T)

                                                                 0
Title                             Fullmetal Alchemist: Brotherhood
Type                                                            TV
Episodes                                                        64
Status                                             Finished Airing
Start airing                                              2009-4-5
End airing                                                2010-7-4
Starting season                                             Spring
Broadcast time                              Sundays at 17:00 (JST)
Producers        Aniplex,Square Enix,Mainichi Broadcasting Syst...
Licensors                            Funimation,Aniplex of America
Studios                                                      Bones
Sources                                                      Manga
Genres           Action,Military,Adventure,Comedy,Drama,Magic,F...
Duration                                           24 min. per

Тестовый набор состоит из первый десяти элементов.

In [7]:
print('Обучающая выборка: \t', ds_anime.shape)
print('Тестовый набор: \t', test_anime.shape)

Обучающая выборка: 	 (1563, 20)
Тестовый набор: 	 (10, 20)


In [8]:
print(ds_anime.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1563 entries, 0 to 1562
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Title            1563 non-null   object 
 1   Type             1563 non-null   object 
 2   Episodes         1563 non-null   object 
 3   Status           1563 non-null   object 
 4   Start airing     1563 non-null   object 
 5   End airing       1563 non-null   object 
 6   Starting season  1563 non-null   object 
 7   Broadcast time   1563 non-null   object 
 8   Producers        1563 non-null   object 
 9   Licensors        1563 non-null   object 
 10  Studios          1563 non-null   object 
 11  Sources          1563 non-null   object 
 12  Genres           1563 non-null   object 
 13  Duration         1563 non-null   object 
 14  Rating           1554 non-null   object 
 15  Score            1563 non-null   float64
 16  Scored by        1563 non-null   int64  
 17  Members       

Корреляция признаков

In [11]:
corr = ds_anime.corr(numeric_only=True)
corr[corr > 0.9].replace({np.nan : ''})

,Score,Scored by,Members,Favorites
Score,1.0,,,
Scored by,,1.0,0.985944,
Members,,0.985944,1.0,
Favorites,,,,1.0


Количество аниме-фильмов, выпущенных за различные года

In [26]:
df = ds_anime.copy()
df['Start airing'] = pd.to_datetime(df['Start airing'], format='%Y-%m-%d', errors='coerce').dt.year
ss = df['Start airing'].value_counts()

print(ss.sort_index())

Start airing
1970.0      1
1971.0      1
1974.0      1
1975.0      1
1976.0      1
1977.0      2
1978.0      6
1979.0      6
1980.0      2
1981.0      4
1982.0      8
1983.0      4
1984.0      5
1985.0      6
1986.0      5
1987.0      6
1988.0     11
1989.0      9
1990.0      9
1991.0      8
1992.0     10
1993.0     12
1994.0     14
1995.0     20
1996.0     18
1997.0     12
1998.0     14
1999.0     24
2000.0     16
2001.0     19
2002.0     28
2003.0     31
2004.0     39
2005.0     51
2006.0     57
2007.0     72
2008.0     75
2009.0     84
2010.0     81
2011.0     95
2012.0     89
2013.0     94
2014.0    114
2015.0    113
2016.0    115
2017.0    110
2018.0     46
Name: count, dtype: int64


Работа с TfidfVectorizer

In [27]:
# Определение списка признаков для обучения модели
feats = ['Title', 'Licensors', 'Studios', 'Sources']

# Вычисление размера обучающей подвыборки (70% от размера исходного набора данных)
train_size = int(0.7 * ds_anime.shape[0])

# Вывод информации о размере исходного набора данных и размере обучающей подвыборки
print('Размер исходного набора: ', len(ds_anime), \
      '\nРазмер обучающей подвыборки: ', train_size)

# Выделение признаков и целевой переменной для обучающей и тестовой выборок
X, y = ds_anime.loc[:, feats], ds_anime['Score']
X_test = test_anime.loc[:, feats]
X_train, X_valid = X.iloc[:train_size, :], X.iloc[train_size:, :]
y_train, y_valid = y.iloc[:train_size], y.iloc[train_size:]

Размер исходного набора:  1563 
Размер обучающей подвыборки:  1094


Создадим три словаря и три матрицы TF-IDF — <i>статистическая мера, используемая для оценки важности слова в контексте документа, являющегося частью коллекции документов или корпуса</i>

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer


# Инициализация TfidfVectorizer с заданными параметрами
vectorizer_title = TfidfVectorizer(min_df=3, max_df=0.3, ngram_range=(1,3))

# Обучение TfidfVectorizer на обучающих данных и вывод размера словаря
vX_train_title = vectorizer_title.fit(X_train['Title'])
print('vX_train_title.vocabulary_: ', len(vX_train_title.vocabulary_))

# Применение обученного TfidfVectorizer к валидационным данным и вывод размера словаря
vX_valid_title = vectorizer_title.fit(X_valid['Title'])
print('vX_train_title.vocabulary_: ', len(vX_valid_title.vocabulary_))

# Применение обученного TfidfVectorizer к тестовым данным и вывод размера словаря
vX_test_title = vectorizer_title.fit(X_test['Title'])
print('vX_train_title.vocabulary_: ', len(vX_test_title.vocabulary_))

# Преобразование обучающих данных с помощью обученного TfidfVectorizer и вывод размера полученной матрицы
X_train_title = vectorizer_title.fit_transform(X_train['Title'])
print('X_train_title.shape: ', X_train_title.shape)

# Преобразование валидационных данных с помощью обученного TfidfVectorizer и вывод размера полученной матрицы
X_valid_title = vectorizer_title.transform(X_valid['Title'])
print('X_valid_title.shape: ', X_valid_title.shape)

# Преобразование тестовых данных с помощью обученного TfidfVectorizer и вывод размера полученной матрицы
X_test_title = vectorizer_title.transform(X_test['Title'])
print('X_test_title.shape: ', X_test_title.shape)

vX_train_title.vocabulary_:  726
vX_train_title.vocabulary_:  235
vX_train_title.vocabulary_:  1
X_train_title.shape:  (1094, 726)
X_valid_title.shape:  (469, 726)
X_test_title.shape:  (10, 726)


Вывод полей тестовой матрицы TF-IDF

In [29]:
vX_test_title.vocabulary_

{'fullmetal': 134,
 'alchemist': 16,
 'fullmetal alchemist': 135,
 'kimi': 270,
 'no': 399,
 'na': 374,
 'wa': 667,
 'kimi no': 271,
 'gintama': 157,
 'steins': 599,
 'gate': 142,
 'steins gate': 600,
 'ginga': 154,
 'eiyuu': 112,
 'densetsu': 94,
 'ginga eiyuu': 155,
 'eiyuu densetsu': 113,
 'ginga eiyuu densetsu': 156,
 'hunter': 208,
 'hunter hunter': 209,
 '2nd': 4,
 'season': 522,
 '2nd season': 5,
 'movie': 362,
 'kanketsu': 257,
 'hen': 187,
 'yo': 694,
 'eien': 111,
 'kanketsu hen': 258,
 'clannad': 74,
 'story': 601,
 'haikyuu': 169,
 'koukou': 288,
 'vs': 666,
 'gakuen': 139,
 'code': 76,
 'geass': 143,
 'hangyaku': 178,
 'lelouch': 307,
 'code geass': 77,
 'geass hangyaku': 144,
 'hangyaku no': 179,
 'no lelouch': 418,
 'code geass hangyaku': 78,
 'geass hangyaku no': 145,
 'hangyaku no lelouch': 180,
 'in': 220,
 'to': 644,
 'second': 523,
 'second season': 524,
 'iii': 219,
 'shouwa': 574,
 'genroku': 146,
 'rakugo': 490,
 'shinjuu': 561,
 'shouwa genroku': 575,
 'genroku 

Повторное создание матриц и словарей, но с символами в качестве параметров

In [30]:
vectorizer_title_ch = TfidfVectorizer(analyzer='char')


vX_train_title_ch = vectorizer_title_ch.fit(X_train['Title'])
print('vX_train_title_ch.vocabulary: ', len(vX_train_title_ch.vocabulary_))

vX_valid_title_ch = vectorizer_title_ch.fit(X_valid['Title'])
print('vX_valid_title_ch.vocabulary_: ', len(vX_valid_title_ch.vocabulary_))

vX_test_title_ch = vectorizer_title_ch.fit(X_test['Title'])
print('vX_test_title_ch.vocabulary_: ', len(vX_test_title_ch.vocabulary_))


X_train_title_ch = vectorizer_title_ch.transform(X_train['Title'])
print('X_train_title_ch.shape: ', X_train_title_ch.shape)

X_valid_title_ch = vectorizer_title_ch.transform(X_valid['Title'])
print('X_valid_title_ch.shape: ', X_valid_title_ch.shape)

X_test_title_ch = vectorizer_title_ch.transform(X_test['Title'])
print('X_test_title_ch.shape: ', X_test_title_ch.shape)

vX_train_title_ch.vocabulary:  68
vX_valid_title_ch.vocabulary_:  59
vX_test_title_ch.vocabulary_:  34
X_train_title_ch.shape:  (1094, 34)
X_valid_title_ch.shape:  (469, 34)
X_test_title_ch.shape:  (10, 34)


Создание матриц с помощью DictVectorizer, для пропущенных значений <i>(например, title или любой другой категориальный признак)</i>

In [31]:
from sklearn.feature_extraction import DictVectorizer


vectorizer_feats = DictVectorizer()

tmp_dict_train = X_train[feats].fillna('-').T.to_dict().values()
tmp_dict_valid = X_valid[feats].fillna('-').T.to_dict().values()
tmp_dict_test = X_test[feats].fillna('-').T.to_dict().values()

X_train_feats = vectorizer_feats.fit_transform(tmp_dict_train)
X_valid_feats = vectorizer_feats.transform(tmp_dict_valid)
X_test_feats = vectorizer_feats.transform(tmp_dict_test)

print(X_train_feats.shape)
print(X_valid_feats.shape)
print(X_test_feats.shape)

(1094, 1358)
(469, 1358)
(10, 1358)


Объединение полученных матриц

In [32]:
X_train_new = scipy.sparse.hstack([X_train_title, X_train_feats, X_train_title_ch])
X_valid_new = scipy.sparse.hstack([X_valid_title, X_valid_feats, X_valid_title_ch])
X_test_new = scipy.sparse.hstack([X_test_title, X_test_feats, X_test_title_ch])

print(X_train_new.shape)
print(X_valid_new.shape)
print(X_test_new.shape)

(1094, 2118)
(469, 2118)
(10, 2118)


In [33]:
%%time
model_1 = Ridge(alpha=.1, random_state=1)
model_1.fit(X_train_new, y_train)

train_preds1 = model_1.predict(X_train_new)
valid_preds1 = model_1.predict(X_valid_new)

print('Ошибка в ходе обучения: ', mean_squared_error(y_train, train_preds1))
print('Ошибка в ходе тестирования: ', mean_squared_error(y_valid, valid_preds1))

Ошибка в ходе обучения:  0.00028957709404417545
Ошибка в ходе тестирования:  0.1688719700526169
CPU times: total: 31.2 ms
Wall time: 24 ms


In [34]:
%%time
model_2 = Ridge(alpha=1.8, random_state=1)
model_2.fit(X_train_new, y_train)

train_preds2 = model_2.predict(X_train_new)
valid_preds2 = model_2.predict(X_valid_new)

print('Ошибка в ходе обучения: ', mean_squared_error(y_train, train_preds2))
print('Ошибка в ходе тестирования: ', mean_squared_error(y_valid, valid_preds2))

Ошибка в ходе обучения:  0.018093116654264225
Ошибка в ходе тестирования:  0.17371673902947063
CPU times: total: 15.6 ms
Wall time: 14 ms


# Вывод: 
Результаты показывают, что model_1 показывает лучшие результаты по сравнению с model_2, так как достигает более низких значений cреднеквадратичной ошибки MSE как на обучающем, так и на валидационном наборе данных. Кроме того, model_1 требует больше времени на обучение по сравнению с model_2.